#####**UNIVERSIDAD AUTONOMA DE CHIHUAHUA**
#####**FACULTAD DE INGENIERIA**
#####**MOBY DICK**
#####**NOMBRE:** JOHANN LOZANO ENRIQUEZ
#####**GRUPO:** 8CC2
#####**MATRICULA:** 338834
#####**DOCENTE:** JESUS ROBERTO LOPEZ SANTILLAN
#####**MATERIA:** DATA SCIENCE
#####***Chihuahua, Chih. 28 de Noviembre del 2023***

In [1]:
from tensorflow import keras

moby_url = "https://www.gutenberg.org/cache/epub/2701/pg2701.txt" # shortcut URL
filepath = keras.utils.get_file("moby.txt", moby_url)
with open(filepath) as f:
  moby_text = f.read()

1276266/1276266 [==============================] - 0s 0us/step


In [2]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([moby_text])

In [3]:
tokenizer.texts_to_sequences(["First"])

[[19, 7, 10, 8, 3]]

In [4]:
tokenizer.sequences_to_texts([[19, 7, 10, 8, 3]])

['f i r s t']

In [5]:
max_id = len(tokenizer.word_index) # number of distinct characters
#dataset_size = tokenizer.document_count # total number of character
dataset_size = sum([x for _,x in tokenizer.word_counts.items()])
print(max_id)
print(dataset_size)
print(type(dataset_size))

77
1238203
<class 'int'>


In [6]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([moby_text])) - 1

In [7]:
import tensorflow as tf
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [8]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [9]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [10]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [11]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

In [12]:
dataset = dataset.prefetch(1)

In [13]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history=model.fit(dataset, epochs=20)
model.save('my_model.h5')

Epoch 1/20
34822/34822 [==============================] - 576s 16ms/step - loss: 1.7377
Epoch 2/20
34822/34822 [==============================] - 520s 15ms/step - loss: 1.6520
Epoch 3/20
34822/34822 [==============================] - 526s 15ms/step - loss: 1.6317
Epoch 4/20
34822/34822 [==============================] - 515s 15ms/step - loss: 1.6210
Epoch 5/20
34822/34822 [==============================] - 521s 15ms/step - loss: 1.6141
Epoch 6/20
34822/34822 [==============================] - 530s 15ms/step - loss: 1.6093
Epoch 7/20
34822/34822 [==============================] - 511s 15ms/step - loss: 1.6063
Epoch 8/20
34822/34822 [==============================] - 513s 15ms/step - loss: 1.6037
Epoch 9/20
34822/34822 [==============================] - 516s 15ms/step - loss: 1.6010
Epoch 10/20
34822/34822 [==============================] - 516s 15ms/step - loss: 1.5988
Epoch 11/20
34822/34822 [==============================] - 518s 15ms/step - loss: 1.5974
Epoch 12/20
34822/34822 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
model = tf.keras.models.load_model('my_model.h5')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         79488     
                                                                 
 gru_1 (GRU)                 (None, None, 128)         99072     
                                                                 
 time_distributed (TimeDist  (None, None, 77)          9933      
 ributed)                                                        
                                                                 
Total params: 188493 (736.30 KB)
Trainable params: 188493 (736.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) - 1
  return tf.one_hot(X, max_id)

In [17]:
X_new = preprocess(["How are yo"])
Y_pred_prob = model.predict(X_new)
Y_pred_classes = np.argmax(Y_pred_prob, axis=-1)

predicted_text = tokenizer.sequences_to_texts(Y_pred_classes +1)[0][-1]
print(predicted_text)

1/1 [==============================] - 1s 597ms/step
u


In [18]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [19]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

In [20]:
print(complete_text("t", temperature=0.2))

1/1 [==============================] - 0s 21ms/step
the steel way started to the deck, and the mast-hea


In [21]:
tf.random.set_seed(42)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
  dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
  dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_length))
  datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)

In [22]:
stateful_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2,batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [23]:
class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [24]:
stateful_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
stateful_model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

Epoch 1/50
348/348 [==============================] - 9s 15ms/step - loss: 2.6847
Epoch 2/50
348/348 [==============================] - 7s 21ms/step - loss: 2.2927
Epoch 3/50
348/348 [==============================] - 5s 15ms/step - loss: 2.1708
Epoch 4/50
348/348 [==============================] - 6s 17ms/step - loss: 2.1025
Epoch 5/50
348/348 [==============================] - 7s 20ms/step - loss: 2.0564
Epoch 6/50
348/348 [==============================] - 5s 16ms/step - loss: 2.0235
Epoch 7/50
348/348 [==============================] - 5s 15ms/step - loss: 1.9956
Epoch 8/50
348/348 [==============================] - 5s 15ms/step - loss: 1.9767
Epoch 9/50
348/348 [==============================] - 7s 19ms/step - loss: 1.9585
Epoch 10/50
348/348 [==============================] - 5s 15ms/step - loss: 1.9428
Epoch 11/50
348/348 [==============================] - 7s 20ms/step - loss: 1.9304
Epoch 12/50
348/348 [==============================] - 7s 19ms/step - loss: 1.9212
Epoch 13/50
3

In [25]:
stateful_model.save('my_model_stateful.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [27]:
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model
tf.random.set_seed(42)
print(complete_text("t"))

1/1 [==============================] - 0s 22ms/step
t the luster of all this jags in the devel parster 
